Setup paths for the notebook

In [1]:
import os
import sys
sys.path.append(os.getcwd() + '/../vic3analyze')


Load the replay data

In [2]:
import parse
import logging
logging.basicConfig(level=logging.DEBUG)

replay_f = '~/.local/share/Paradox Interactive/Victoria 3/save games/autosave.v3'
replay_f = os.path.expanduser(replay_f)

replay_data = parse.parse(replay_f)

print("Done loading replay")

INFO:root:PyPy detected, attempting parsing using it
INFO:root:Reading savefile
INFO:root:Parsing save data into AST
INFO:root:Parsing AST into data structure
INFO:root:Reading back data from PyPy


Done loading replay


In [3]:

for key, val in replay_data.items():
    if isinstance(val, int) or isinstance(val, str):
        print(key, val)
    elif isinstance(val, dict):
        print(key, list(val.keys())[0:20])
    else:
        print(key, type(val))

meta_data ['save_game_version', 'version', 'game_date', 'real_date', 'dlcs', 'game_rules', 'achievement_eligibility', 'ironman', 'number_of_players']
ironman ['ironman', 'date', 'save_game', 'save_interval', 'storage']
playthrough_id fe5b7b6f-a158-4af9-b1f5-e1a82de7f80d
date <class 'datetime.datetime'>
seed 822048903
seed_count 302374
speed 4
first_start no
counters ['command', 'tick', 'week', 'province_owner', 'province_theater', 'state_provinces', 'wars', 'diplomatic_play_participants', 'markets', 'create_destroy_markets', 'fully_executed_commands', 'modifiers']
variables ['data']
provinces [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
pops ['database', 'ig_update_index', 'popgrowth_update_index', 'popdiscrimination_update_index', 'poployalties_update_index', 'pop_weight_randomizer', 'dead_objects', 'wealth_initialized']
country_manager ['database', 'dead_countries']
states ['database', 'state_region_to_state_array', 'dead_objects']
interest_groups ['database

In [4]:
from textwrap import dedent

ml = replay_data

def get_sampledata(d_obj):
    if 'channels' not in d_obj:
        return 0
    return d_obj['channels'][0]['values'][0]

print("===Country DB keys===")
country_db = ml['country_manager']['database']
for key in country_db[1].keys():
    print(key)



print("===Country ranks===")
for c in ml['country_rankings']['country_rankings']:
    print(c)
    

print("===States db===")  
states_db = ml['states']['database']
for key in states_db[1].keys():
    print(key)

===Country DB keys===
is_main_tag
definition
government
infamy
ruler
heir
capital
budget
gdp
prestige
avgsoltrend
tax_level
country_type
variables
cultures
religion
pop_statistics
pop_radicals_and_loyalists_statistics
states
outliner_data
taxed_goods
declared_interests
construction_queue
default_on_action_counter
convoys_produced
convoys_from_subjects
diplomatic_pacts_counter
counters
===Country ranks===
{'rank': 'great_power', 'target': 'great_power', 'prestige': 1439, 'score': 1, 'country': 1}
{'rank': 'great_power', 'target': 'great_power', 'prestige': 865, 'score': 3, 'country': 3}
{'rank': 'great_power', 'target': 'great_power', 'prestige': 1118, 'score': 2, 'country': 4}
{'rank': 'great_power', 'target': 'great_power', 'prestige': 408, 'score': 5, 'country': 5}
{'rank': 'great_power', 'target': 'great_power', 'prestige': 497, 'score': 4, 'country': 8}
{'rank': 'great_power', 'target': 'great_power', 'prestige': 299, 'score': 6, 'country': 9}
{'rank': 'unrecognized_major_power', '

In [5]:

for k, v in ml['country_manager']['database'].items():
    if isinstance(v, dict):
        tag = v['definition']
        gdp = get_sampledata(v['gdp'])
        prestige = get_sampledata(v['prestige'])
        sol = get_sampledata(v['avgsoltrend'])
        ps = v['pop_statistics']
        pop = get_sampledata(ps['population_trend'])
        rad = get_sampledata(ps['radical_trend'])
        loy = get_sampledata(ps['loyalist_trend'])
        bud = v['budget']
        treasury = bud['money']
        investement = bud.get('investment_pool',0)
        credit = bud['credit']
        n_states = len(v['states'])
        print(dedent(f"""\
            -- {k:3.0f} {tag} --
            GDP: {gdp:10.0f} Prestige: {prestige:6.0f} SoL: {sol:3.1f}
            pop: {pop:9.0f} radicals: {rad:9.0f} loyalists: {loy:9.0f}
            treasury: {treasury:9.0f} investement: {investement:9.0f} credit: {credit:9.0f}
            states: {n_states}
            """))

-- 16777216 NIC --
GDP:          0 Prestige:      0 SoL: 0.0
pop:         0 radicals:         0 loyalists:         0
treasury:         0 investement:         0 credit:    155667
states: 1

--   1 GBR --
GDP:    1095812 Prestige:   1410 SoL: 14.1
pop:  26068859 radicals:      5391 loyalists:         0
treasury:   5883324 investement:    811229 credit:  14816986
states: 28

--   3 RUS --
GDP:     914442 Prestige:    860 SoL: 9.4
pop:  58813704 radicals:         0 loyalists:         0
treasury:   4673488 investement:         0 credit:  13466249
states: 54

--   4 FRA --
GDP:     920021 Prestige:   1113 SoL: 11.4
pop:  33894723 radicals:      4428 loyalists:         0
treasury:   5206399 investement:    532533 credit:  13997323
states: 26

--   5 PRU --
GDP:     354775 Prestige:    404 SoL: 11.5
pop:  13740708 radicals:        21 loyalists:         0
treasury:   1737568 investement:         0 credit:   5232864
states: 11

--   8 AUS --
GDP:     533221 Prestige:    491 SoL: 11.4
pop:  34695

In [10]:
import game_static
goods_data = game_static.get_config_file('common/goods/00_goods.txt')
goods_lookup = list(goods_data.keys())

from collections import defaultdict
mkt_goods = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))



for b_id, b in ml['building_manager']['database'].items():
    if isinstance(b, str):
        continue
    goods_in = b.get('input_goods', {}).get('goods',{})
    goods_out = b.get('output_goods', {}).get('goods',{})
    market = ml['states']['database'][b['state']]['market']
    for good, amount in goods_in.items():
        goodname = goods_lookup[good]
        mkt_goods[market][goodname]['bld_demand'] += amount
    for good, amount in goods_out.items():
        goodname = goods_lookup[good]
        mkt_goods[market][goodname]['bld_supply'] += amount

In [15]:
for k,v in ml['trade_route_manager']['database'].items():
    if v['direction'] == 'import':
        mkt_goods[v['source']][v['goods']]['mkt_import'] += v.get('traded',0)
        mkt_goods[v['target']][v['goods']]['mkt_export'] += v.get('traded',0)
    elif v['direction'] == 'export':
        mkt_goods[v['target']][v['goods']]['mkt_import'] += v.get('traded',0)
        mkt_goods[v['source']][v['goods']]['mkt_export'] += v.get('traded',0)
    

In [8]:
import re
game_root = os.path.expanduser('~/.steam/steam/steamapps/common/Victoria 3/game')

with open(os.path.join(game_root, 'common/goods/00_goods.txt')) as f:
    rawdata =f.read()
    
rawdata = re.sub("#.*","",rawdata).strip()
print(rawdata.encode('utf-8'))

b'\xef\xbb\xbf\n\n\n\n\n\n\n\nammunition = {\n\ttexture = "gfx/interface/icons/goods_icons/ammunition.dds"\n\tcost = 50\n\tcategory = military\n\t\n\tprestige_factor = 5\n}\n\nsmall_arms = {\n\ttexture = "gfx/interface/icons/goods_icons/small_arms.dds"\n\tcost = 60\n\tcategory = military\n\t\n\tprestige_factor = 5\n}\n\nartillery = {\n\ttexture = "gfx/interface/icons/goods_icons/artillery.dds"\n\tcost = 70\n\tcategory = military\n\t\n\tprestige_factor = 5\n\ttraded_quantity = 7\n\tconvoy_cost_multiplier = 1.5\n}\n\ntanks = {\n\ttexture = "gfx/interface/icons/goods_icons/tanks.dds"\n\tcost = 80\n\tcategory = military\n\t\n\tprestige_factor = 10\n\ttraded_quantity = 6\n\tconvoy_cost_multiplier = 2.0\n}\n\naeroplanes = {\n\ttexture = "gfx/interface/icons/goods_icons/aeroplanes.dds"\n\tcost = 80\n\tcategory = military\n\t\n\tprestige_factor = 10\n\ttraded_quantity = 6\n\tconvoy_cost_multiplier = 2.0\n}\n\nmanowars = {\n\ttexture = "gfx/interface/icons/goods_icons/man_o_wars.dds"\n\tcost = 